<a href="https://colab.research.google.com/github/slgodwyn/GPT-neo/blob/colab/Copy_of_GPT_web.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!sudo apt-get update
!sudo apt-get install nvidia-cuda-toolkit

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [697 B]
Hit:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:7 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:9 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ Packages [55.5 kB]
Hit:10 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:12 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:13 http://ppa.launchpad.net/cran/

In [ ]:
!pip install torch torchvision torchaudio torchtext
!pip install -q gradio
!pip install aitextgen

     |████████████████████████████████| 1.9MB 6.6MB/s 
     |████████████████████████████████| 1.1MB 6.3MB/s 
     |████████████████████████████████| 215kB 17.3MB/s 
     |████████████████████████████████| 1.9MB 18.2MB/s 
     |████████████████████████████████| 3.2MB 41.4MB/s 
     |████████████████████████████████| 71kB 8.1MB/s 
     |████████████████████████████████| 962kB 60.2MB/s 
     |████████████████████████████████| 573kB 4.1MB/s 
     |████████████████████████████████| 2.1MB 47.9MB/s 
     |████████████████████████████████| 92kB 9.8MB/s 
     |████████████████████████████████| 808kB 51.8MB/s 
     |████████████████████████████████| 3.3MB 56.8MB/s 
     |████████████████████████████████| 901kB 56.8MB/s 
     |████████████████████████████████| 829kB 60.9MB/s 
     |████████████████████████████████| 276kB 52.0MB/s 
     |████████████████████████████████| 645kB 63.1MB/s 
     |████████████████████████████████| 112kB 58.5MB/s 
     |████████████████████████████████| 1.3MB 58.3MB/s 

In [ ]:
!pip install cloud-tpu-client==0.10 https://storage.googleapis.com/tpu-pytorch/wheels/torch_xla-1.8.1-cp37-cp37m-linux_x86_64.whl

     |████████████████████████████████| 145.0MB 41kB/s 
     |████████████████████████████████| 61kB 3.0MB/s 
ERROR: earthengine-api 0.1.260 has requirement google-api-python-client<2,>=1.12.1, but you'll have google-api-python-client 1.8.0 which is incompatible.
  Found existing installation: google-api-python-client 1.12.8
    Uninstalling google-api-python-client-1.12.8:
      Successfully uninstalled google-api-python-client-1.12.8


In [ ]:
import torch

# imports the torch_xla package
import torch_xla
import torch_xla.core.xla_model as xm
import gradio as gr
from aitextgen import aitextgen

In [ ]:
dev = xm.xla_device()
t1 = torch.ones(3, 3, device = dev)
print(t1)

tensor([[1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.]], device='xla:1')


In [ ]:
second_dev = xm.xla_device(n=2, devkind='TPU')
t2 = torch.zeros(3, 3, device = second_dev)
print(t2)

tensor([[0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.]], device='xla:2')


In [ ]:
a = torch.randn(2, 2, device = dev)
b = torch.randn(2, 2, device = dev)
print(a + b)
print(b * 2)
print(torch.matmul(a, b))

tensor([[ 0.9285, -1.1517],
        [ 0.4875,  1.0669]], device='xla:1')
tensor([[ 1.6515,  0.6532],
        [ 1.8291, -0.5493]], device='xla:1')
tensor([[-1.2652,  0.4401],
        [ 0.8757, -0.5096]], device='xla:1')


In [ ]:
filters = torch.randn(33, 16, 3, device = dev)
inputs = torch.randn(20, 16, 50, device = dev)
torch.nn.functional.conv1d(inputs, filters)

tensor([[[ 7.4347e+00, -9.3358e+00,  2.8346e+00,  ...,  4.8745e-01,
          -1.1800e+01,  2.3509e+00],
         [-5.6661e+00, -9.2697e+00, -2.7056e+00,  ..., -5.8482e+00,
          -7.8100e+00,  4.6295e+00],
         [-4.4290e+00, -6.9281e+00,  2.1860e+00,  ...,  1.2434e+01,
          -7.8181e+00, -5.3043e-01],
         ...,
         [ 1.6844e+01,  9.5925e+00, -7.4014e+00,  ..., -3.5882e+00,
           8.9396e-01, -1.1485e+01],
         [-1.2405e+01, -9.6386e+00,  5.4843e+00,  ...,  3.7790e+00,
          -3.0254e+00, -1.1809e+00],
         [ 4.5209e+00, -1.1931e+01, -2.5749e+00,  ...,  5.3200e+00,
          -2.9704e+00,  1.5700e+00]],

        [[ 2.5035e+00,  8.3513e+00, -4.9031e+00,  ...,  1.3565e+01,
          -9.2602e+00,  2.9704e+00],
         [-3.6338e+00,  5.2198e+00,  4.7569e+00,  ...,  2.1375e+00,
          -2.6774e+00, -1.1080e+01],
         [ 8.0821e-01, -4.5582e+00, -1.8723e+00,  ..., -4.4451e+00,
           1.5516e+00, -1.0374e+01],
         ...,
         [ 1.1265e+00,  3

In [ ]:
import gradio as gr
from aitextgen import aitextgen

In [ ]:
ai = aitextgen(model="EleutherAI/gpt-neo-2.7B")


INFO:aitextgen:Downloading EleutherAI/gpt-neo-2.7B model to /aitextgen.


INFO:aitextgen:Using the tokenizer for EleutherAI/gpt-neo-2.7B.


loading file https://huggingface.co/EleutherAI/gpt-neo-2.7B/resolve/main/vocab.json from cache at aitextgen/d4455fdc7c8e2bcf94a0bfe134b748a93c37ecadb7b8f6b0eb508ffdd433a61e.a1b97b074a5ac71fad0544c8abc1b3581803d73832476184bde6cff06a67b6bb
loading file https://huggingface.co/EleutherAI/gpt-neo-2.7B/resolve/main/merges.txt from cache at aitextgen/5660be25091706bde0cfb60f17ae72c7a2aa40223d68954d4d8ffd1fc6995643.f5b91da9e34259b8f4d88dbc97c740667a0e8430b96314460cdb04e86d4fc435
loading file https://huggingface.co/EleutherAI/gpt-neo-2.7B/resolve/main/tokenizer.json from cache at None
loading file https://huggingface.co/EleutherAI/gpt-neo-2.7B/resolve/main/added_tokens.json from cache at None
loading file https://huggingface.co/EleutherAI/gpt-neo-2.7B/resolve/main/special_tokens_map.json from cache at aitextgen/953b5ce47652cf8b6e945b3570bfa7621164c337e05419b954dbe0a4d16a7480.3ae9ae72462581d20e36bc528e9c47bb30cd671bb21add40ca0b24a0be9fac22
loading file https://huggingface.co/EleutherAI/gpt-neo-2

INFO:aitextgen:GPTNeo loaded with 2651M parameters.


In [ ]:
def ai_text(input):
  res = ai.generate_one(max_length = 1000, prompt = inp, no_repeat_ngram_size = 3) #repetition_penalty = 1.9)
  #print(type(generated_text))
  return res

In [1]:
output_text = gr.outputs.Textbox()
gr.Interface(ai_text,"textbox", output_text, title="AI Generated Blog with GPT-Neo",
             description="Enter a topic sentence below and press submit", layout='vertical', allow_screenshot=False, interpretation="Default", allow_flagging=False).launch(share=True)

NameError: ignored

In [ ]:
!pip freeze > /users/stevengodwyn/documents/requirements.txt

/bin/bash: /users/stevengodwyn/documents/requirements.txt: No such file or directory
